###Chromeインストール

In [ ]:
%%shell

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

In [ ]:
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

!apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
!apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
!apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

###selenium インストール

In [ ]:
!pip install selenium

###edinetから過去１年分の四半期決算データをダウンロード

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(10)

target_url = "https://disclosure2.edinet-fsa.go.jp/"
driver.get(target_url)
time.sleep(3)

element = driver.find_element(By.ID, "W0018vD_KEYWORD")
element.send_keys("エディア")
time.sleep(3)

select_elem = driver.find_element(By.ID, "W0018vD_KIKAN")
Select(select_elem).select_by_visible_text("過去１年")
time.sleep(3)

button = driver.find_element(By.ID, "W0018BTNBTN_SEARCH")
button.click()
time.sleep(5)

table = driver.find_element(By.ID, "GridContainerTbl")
print(table)
trs = table.find_elements(By.TAG_NAME, 'tr')
for tr in trs[1:]:
    tds = tr.find_elements(By.TAG_NAME, 'td')
    print(tds[0].text)
    print(tds[1].text)
    tds[6].find_element(By.TAG_NAME, 'a').click()
    time.sleep(3)

<selenium.webdriver.remote.webelement.WebElement (session="2e2ad68a294abffcd8e17c089f32cbfe", element="0b55b9ad-f3d2-4c93-9ab8-826aeef016a3")>
2023/01/13 15:04
四半期報告書－第24期第3四半期(2022/09/01－2022/11/30)
2022/10/14 15:18
四半期報告書－第24期第2四半期(2022/06/01－2022/08/31)
2022/07/15 15:38
四半期報告書－第24期第1四半期(2022/03/01－2022/05/31)
2022/05/25 16:43
有価証券報告書－第23期(2021/03/01－2022/02/28)


###zipファイル解凍

In [ ]:
import os
import zipfile

path = "."

# ファイル一覧を取得
files = os.listdir(path)
for file in files:
  if file.find('.zip') == -1:
    continue
  
  try:
      # zipファイル解凍
      extract_dir_name = file.replace('.zip', '')
      with zipfile.ZipFile(file) as existing_zip:
          existing_zip.extractall(extract_dir_name)
  except:
    continue

###４半期短期決算の営業利益データを取得

####・zip解凍フォルダからhtmファイルを取得 → 本年度と前年度の営業利益を取得

In [ ]:
import glob

def get_values(xbrl_dir_name):
  # zip解凍フォルダからhtmファイルを取得
  files = os.listdir(xbrl_dir_name)
  dir_name = [file for file in files if os.path.isdir(xbrl_dir_name + '/' + file)][0]

  dir_path = f'{xbrl_dir_name}/{dir_name}'
  file_dir = f'{dir_path}/XBRL/PublicDoc/*'
  files = glob.glob(file_dir)
  html_files = [file for file in files if file.find('.htm') > 0]
  return get_html_values(html_files)

In [ ]:
from bs4 import BeautifulSoup

def get_html_values(html_files):
  term = ""
  value = ""
  pre_term = ""
  pre_value = ""
  for html_file in html_files:
    # 本年度と前年度の営業利益を取得
    try:
      bs = BeautifulSoup(open(html_file, 'r').read(), 'lxml')
      if bs.find('p').text.find('四半期連結損益計算書') != -1:
        trs = bs.find_all('table')[0].find_all('tr')
        for tr in trs:
          if tr.text.find("四半期連結累計期間") > -1:
            terms = tr.text.strip().split("\n")
            terms.remove('')
            term = terms[-1]
            pre_term = terms[0]
          if tr.text.find("営業利益") > -1:
            values = tr.text.strip().split("\n")
            values.remove('')
            value = values[-1]
            pre_value = values[1]
    except:
      continue
  return (term, value, pre_term, pre_value)

In [ ]:
import pandas as pd

path = "."
files = os.listdir(path)
extract_dir_names = [file_name for file_name in files if file_name.find('.zip') == -1 and file_name.find('Xbrl') != -1]

dic ={}
for extract_dir_name in extract_dir_names:
  # zip解凍フォルダからhtmファイルを取得 → 本年度と前年度の営業利益を取得
  term, value, pre_term, pre_value = get_values(extract_dir_name)

  if term == "":
    continue
 
  dic[term] = {'value': value, 'pre_value': pre_value}

dic 

{'当第１四半期連結累計期間(自\xa02022年３月１日\u3000至\xa02022年５月31日)': {'value': '46,224',
  'pre_value': '25,708'},
 '当第２四半期連結累計期間(自\xa02022年３月１日\u3000至\xa02022年８月31日)': {'value': '79,976',
  'pre_value': '57,862'},
 '当第３四半期連結累計期間(自\xa02022年３月１日\u3000至\xa02022年11月30日)': {'value': '102,712',
  'pre_value': '94,109'}}

###取得した４半期決算の営業利益をグラフ化

In [23]:
import re

# 取得したデータをデータフレーム化
df = pd.DataFrame(dic.values(), index=dic.keys())
df.columns = ['value', 'pre_value']

# 営業利益のカラムを数値化
df = df.replace(',', '', regex=True)
df["value"] = df['value'].astype('float')
df["pre_value"] = df['pre_value'].astype('float')

# 四半期の名前を正規化
df['title'] = [re.search('第(\d*)四半期', x).group(0) for x in df.index]
df.index = df['title']

del df['title'] #df = df.drop('title', axis=1)

# ソートして確認
df = df.sort_index()

df_total = pd.DataFrame({'value': [df['value'].sum()], 'pre_value': [df['pre_value'].sum()]}, index=["通期"])
df = pd.concat([df, df_total])

df

,value,pre_value
第１四半期,46224.0,25708.0
第２四半期,79976.0,57862.0
第３四半期,102712.0,94109.0
通期,228912.0,177679.0


In [25]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='前年度', x=df.index, y=df['pre_value']),
    go.Bar(name='本年度', x=df.index, y=df['value'])
])

fig.update_layout(barmode='group')
fig.show()